In [ ]:
from datasets import Dataset, DatasetDict
import pandas as pd
import numpy as np
import mord as m
import os
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import mutual_info_regression, SelectKBest, f_classif, VarianceThreshold
from sklearn.metrics import mean_absolute_error, mean_squared_error, root_mean_squared_error, r2_score, cohen_kappa_score
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor, HistGradientBoostingRegressor, GradientBoostingRegressor
from sklearn.svm import SVR, SVC
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV
from functools import partial

# Definição do Tipo de Análise:
# Valores de 0 a 4: Competências do ENEM (C1 a C5)
# Valor 5: Nota final (soma das competências)
REFERENCE_CONCEPT = 4

# Carregamento do dataset 




In [ ]:
DATASET_PATH = "../corpus/"
DATASET_NAME = "-00000-of-00001.parquet"
DIVISIONS = ("train", "validation", "test")
def target_dataset_path(target: str):
    if target in DIVISIONS:
        return DATASET_PATH + target + DATASET_NAME
    else:
        raise ValueError("ERROR: Invalid target for dataset.")

datasets_dict = {}

for division in DIVISIONS:
    file_path = target_dataset_path(division)
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"Arquivo não encontrado: {file_path}")
    df = pd.read_parquet(file_path, engine='pyarrow')
    datasets_dict[division] = Dataset.from_pandas(df)

dataset_filtrado = DatasetDict(datasets_dict)

In [ ]:
# Mapeamento de notas para competências (não aplicável à nota final)
grade_mapping = {
    0: 0,
    40: 1,
    80: 2,
    120: 3,
    160: 4,
    200: 5,
}

def create_label(row):
  if REFERENCE_CONCEPT == 5:
    return {"label": row["grades"][-1]}
  else:
    grade = row["grades"][REFERENCE_CONCEPT]
    return {"label": grade_mapping[grade]}

dataset = dataset_filtrado.map(create_label)

# Carregamento das Métricas Extraídas com o NILC-METRIX


In [ ]:
# Carregamento das métricas NILC-Metrix
nome_arquivo = "metricas_redacoes_nilcmetrix.csv"
nilc_df = pd.read_csv(nome_arquivo)

nilc_df = nilc_df.rename(columns={"id_texto": "id"})

# Tirar 'redacao_' e adicionar '.html' no arquivo .csv na coluna dos IDs
nilc_df["id"] = nilc_df["id"].str.replace("redacao_", "") + ".html"

# Merge

In [ ]:
train_df = dataset["train"].to_pandas()
val_df = dataset["validation"].to_pandas()
test_df = dataset["test"].to_pandas()

# Fazer o merge com as métricas NILC
dados_train = pd.merge(train_df, nilc_df, on="id", how="left")
dados_val = pd.merge(val_df, nilc_df, on="id", how="left")
dados_test = pd.merge(test_df, nilc_df, on="id", how="left")

In [ ]:
print(dados_train.isnull().sum()) # Treino: 37 redações sem métricas
print(dados_val.isnull().sum()) # Validação: 9 redações sem métricas
print(dados_test.isnull().sum()) # Teste: 9 redações sem métricas

# Limpeza dos Dados

In [ ]:
metric_cols = nilc_df.columns.drop('id')

dados_train_clean = dados_train.dropna(subset=metric_cols)  # Remove as 37
dados_val_clean = dados_val.dropna(subset=metric_cols)  # Remove as 9
dados_test_clean = dados_test.dropna(subset=metric_cols)  # Remove as 9

In [ ]:
# Agora só ficaram as redações que possuem métricas correspondentes do NILC
print(dados_train_clean.isnull().sum())
print(dados_val_clean.isnull().sum())
print(dados_test_clean.isnull().sum())

# Função para Calcular a Acurácia do ENEM

In [ ]:
def enem_accuracy_score(true_values, predicted_values):
    """Calcula acurácia no padrão ENEM (diferença <= 80 pontos)"""
    if REFERENCE_CONCEPT == 5:
      limite_pontos = 80
    else:
      limite_pontos = 2
    assert len(true_values) == len(predicted_values), "Mismatched length between true and predicted values."  # Verifica se cada valor predito tem um correspondente pra calcular a diferença

    non_divergent_count = sum([1 for t, p in zip(true_values, predicted_values) if abs(t - p) <= limite_pontos])

    return non_divergent_count / len(true_values)

# Função regression_report inspirada na classification_report

In [ ]:
def regression_report(y_true, y_pred):
    """Relatório completo de métricas de regressão"""
    r2 = r2_score(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    enem_acc = enem_accuracy_score(y_true, y_pred)

    # QWK precisa transformar as notas em inteiros
    y_true_rounded = np.round(y_true).astype(int)
    y_pred_rounded = np.round(y_pred).astype(int)

    qwk = cohen_kappa_score(y_true_rounded, y_pred_rounded, weights="quadratic")

    print("Regression Report:")
    print(f"R² Score: {r2:.4f}")
    print(f"Mean Squared Error (MSE): {mse:.4f}")
    print(f"Mean Absolute Error (MAE): {mae:.4f}")
    print(f"Root Mean Squared Error (RMSE): {rmse:.4f}")
    print(f"ENEM Accuracy Score: {enem_acc:.2f}")
    print(f"Quadratic Weighted Kappa (QWK): {qwk:.4f}")
    print()

# Pré-Processamento

In [ ]:
# Pré-processamento das métricas NILC
X_train = dados_train_clean[metric_cols].values
X_test = dados_test_clean[metric_cols].values
y_train = np.array(dados_train_clean["label"]).reshape(-1)
y_test = np.array(dados_test_clean["label"]).reshape(-1)

# Pipeline para regressão
preprocessador = make_pipeline(
    VarianceThreshold(),
    StandardScaler(),
    SelectKBest(score_func=partial(mutual_info_regression, random_state=1), k=80),
)
X_train_transf = preprocessador.fit_transform(X_train, y_train)

# Pipeline para classificação (apenas para competências)
if REFERENCE_CONCEPT != 5:
    preprocessador_clf = make_pipeline(
        VarianceThreshold(),
        StandardScaler(),
        SelectKBest(score_func=f_classif, k=80)
    )
    X_train_clf = preprocessador_clf.fit_transform(X_train, y_train)

# Modelos

In [ ]:
# Dicionário de modelos
modelos = {
    # Modelos de Regressão
    "Lasso": Lasso(),
    "Regressao Linear": LinearRegression(),
    "Ridge": Ridge(),
    "Random Forest Regression": RandomForestRegressor(random_state=1),
    "HistGradientBoostingRegressor": HistGradientBoostingRegressor(random_state=1),
    "SVR": SVR(),
    "MLP": MLPRegressor(random_state=1, max_iter=1000),
    "KNeighborsRegressor": KNeighborsRegressor(),
    "DecisionTreeRegressor": DecisionTreeRegressor(random_state=1),
    "GradientBoostingRegressor": GradientBoostingRegressor(random_state=1),

    # Modelos de regressão ordinal (apenas para competências)
    "Ordinal Regression LogisticAT": m.LogisticAT(alpha=1.0),
    "Ordinal Regression LogisticIT": m.LogisticIT(alpha=1.0),
    "Ordinal Regression OrdinalRidge": m.OrdinalRidge(),
    "Least Absolute Deviation (LAD)": m.LAD(random_state=1) # É como uma regressão linear, mas utiliza o erro absoluto em vez do erro quadrático
}

In [ ]:
print("Tamanho de X_train_transf:", X_train_transf.shape)
print("Número de exemplos (amostras):", X_train_transf.shape[0])
print("Número de atributos (features):", X_train_transf.shape[1])
print("Tamanho de y_train:", len(y_train))
print("Tamanho de y_test:", len(y_test))

# Treinamento e Avaliação

In [ ]:
print(f"++++++++++++ ANÁLISE DA {'NOTA FINAL' if REFERENCE_CONCEPT == 5 else f'COMPETÊNCIA {REFERENCE_CONCEPT + 1}'} ++++++++++++")

for nome, modelo in modelos.items():
  if REFERENCE_CONCEPT == 5 and (nome.startswith("Ordinal Regression") or nome == "Least Absolute Deviation (LAD)"):
        continue  # Pula esses modelos se estiver avaliando a nota final porque mord não funciona sem o mapeamento

  # Treinamento único para todos os modelos, pois os arrays já são densos
  modelo.fit(X_train_transf, y_train)
  y_pred = modelo.predict(preprocessador.transform(X_test))

  # Primeiro imprime o regression_report
  # Depois arredonda os valores previstos para int e imprime o classification_report
  print(f"****  Modelo: {nome} ****")
  regression_report(y_test, y_pred)

  if REFERENCE_CONCEPT != 5:
        y_pred_round = np.round(y_pred).astype(int)
        print("Classification Report:")
        print(classification_report(y_test, y_pred_round, zero_division=0))
        print("Confusion Matrix:")
        print(confusion_matrix(y_test, y_pred_round))
        print()

# Classificação (Somente para Competências)


SVC é a implementação do SVM para problemas de classificação


In [ ]:
if REFERENCE_CONCEPT != 5:
  print("Otimização de SVC para Classificação:")
  param_grid = {'kernel': ['linear', 'rbf', 'poly'], 'C': [0.1, 1, 10, 100]}
  grid_search = GridSearchCV(SVC(), param_grid, cv=5, n_jobs=-1, verbose=1)
  grid_search.fit(X_train_clf, y_train)

  y_pred_clf = grid_search.predict(preprocessador_clf.transform(X_test))

  qwk_clf = cohen_kappa_score(y_test, y_pred_clf, weights="quadratic")
  print(f"Quadratic Weighted Kappa (QWK): {qwk_clf:.4f}")

  rmse_clf = root_mean_squared_error(y_test, y_pred_clf)
  print(f"Root Mean Squared Error (RMSE): {rmse_clf:.4f}")

  enem_acc_clf = enem_accuracy_score(y_test, y_pred_clf)
  print(f"ENEM Accuracy Score: {enem_acc_clf:.2f}")

  print("\nMelhores parâmetros:", grid_search.best_params_)
  print("\nClassification Report:")
  print(classification_report(y_test, y_pred_clf))